# __Age of Empires II aoe2recs.com API modules__

## __Define functions__
1. **Database** - Setup database connection strings and variables. Might have to change based on OS (Linux/Win)
2. **Fetch match details** - Call API and fetch match details

### __1. Database :__

In [50]:
def setup_mongo_conn():
    
    client = MongoClient(os.environ['mongopath'])    
    return(client)

### __2. Read/delete/update matches :__

In [ ]:
def insert_match_mongo(match_data, db_conn):
    
    db_conn.aoe2.matches.insert_one(match_data) 
    
def fetch_latest_match_id_aoe2recs(db_conn):
    
    match_id = db_conn.aoe2.matches.find_one(sort=[("id", -1)])['id']
    return(match_id)

def fetch_missing_match_ids(db_conn):
    
    # Fetch all match ids
    ids = []
    for i in db_conn.aoe2.matches.find({'id':{'$gt': 0}}).sort('id'):
        ids.append(int(i['id']))
        
    # All possible match ids
    all_ids = list(np.array(range(1,fetch_latest_match_id_aoe2recs(db_conn))))
    
    # missing matches
    miss_ids = set(all_ids).difference(ids)
    
    #return
    return(miss_ids)

def fetch_missing_match_ids(db_conn):
    
    # Fetch all match ids
    ids = []
    for i in db_conn.aoe2.matches.find({'id':{'$gt': 0}}).sort('id'):
        ids.append(int(i['id']))
        
    # All possible match ids
    all_ids = list(np.array(range(1,fetch_latest_match_id_aoe2recs(db_conn))))
    
    # missing matches
    miss_ids = set(all_ids).difference(ids)
    
    #return
    return(miss_ids)

### __2. Fetch match details:__

In [45]:
def fetch_all_match_details_aoe2recs(match_id,query):
    
    query = query.replace('$$__$$',str(match_id))
    url = 'https://aoe2recs.com/api'
    res = requests.post(url, json={'query': query}).text
    res = json.loads(res)['data']['match']
    
    if res is None:
        res = {'id': -999}
        
    return(res)

In [ ]:
def fetch_matches_iterate_aoe2recs(db_conn, collect = 10, type = 'new', clean_empty = True, cores = 4):
    
    # CLean empty entries
    if clean_empty:
        db_conn.aoe2.matches.delete_many({"id": -999})
    
    # Function to fetch match details and insert to mongoDB
    def parallel_fetch(match_id,query):   
        
        db_conn = MongoClient(os.environ['mongopath'])  
        match_data = fetch_all_match_details_aoe2recs(match_id,query)
        insert_match_mongo(match_data,db_conn)
    
    # Read query
    with open('/Projects/AgeofEmpires2_Voobly/Input/aoe2recs_match_data_query.txt', "r", encoding="utf-8") as text:
        query = text.read()
        
    if type == 'new':
         # Find last fetched match ID
        start = fetch_latest_match_id_aoe2recs(db_conn) + 1

        match_list = list(range(start,start+collect))
    elif type == 'missing':
                
        match_list = fetch_missing_match_ids(db_conn)
        
    inputs = tqdm(match_list,miniters = 1,bar_format='{desc:<5.5}{percentage:3.0f}%|{bar:60}{r_bar}')
    
    # Parallel fetch matches
    if __name__ == "__main__":
        processed_list = Parallel(n_jobs=cores)(delayed(parallel_fetch)(i,query) for i in inputs)
    
    print('\nDONE !')
    print('\nAOE2 MATCHES DOWNLOADED : ' + str(collect))   